In [ ]:
"""
Code to process the XYZ file produced by open3dqsar into pandas
The ASCII did not have grid coords only the energy values
The param for the format keyword in export is 'XYZ'
Run this code and specify the file name in the fin line
Last few cells are the beginnings of a function to process the entire directory

###############################
import type=dependent file="C:\data\Datasets.Py-CoMFA\07.THERM\therm_binding_values.txt"

##########################
Below is the command for open3dqsar to generate the MIF for python
##########################

import type=mol2 file="C:\data\comb_therm_train.mol2"

box step=1.0

calc_field type=VDW

calc_field type=MM_ELE

export type=OBJECT_FIELD file=therm format=XYZ 

 


"""

In [1]:
import os
import sys
import pandas as pd
import numpy as  np
import time
import os
import shutil

In [2]:
def set_globals():
    global open3D_directory
    global dataset_name_num
    global MIF_nodes 
    global xyz_offset
    open3D_directory = "C:\open3dtools"
    dataset_name_num = "02.AchE"
    MIF_nodes = [26,29,30]
    xyz_offset = [-12,50,53]
    

In [51]:
def agrd_to_array(entry_path):
    fin = open(entry_path, "rt")
    fout = open("out.txt", "wt")

    for line in fin:
        fout.write(' '.join(line.split()))
        fout.write("\n")

    fin.close()
    fout.close()  
    npdata = np.loadtxt("out.txt",skiprows=5,delimiter=" ")
    MIF = np.zeros(MIF_nodes)
    for line in npdata:
        
        x,y,z,target = line
        x -= xyz_offset[0]
        y -= xyz_offset[1]
        z -= xyz_offset[2]
        MIF[int(x),int(y),int(z)] = target
    dataset_name = entry_path[15:].split("_")[0]
    number = entry_path[:-5].split("-")[-1]
    field = entry_path.split("_")[1]
    new_path =str(number+"_"+field+'.npy')    
    np.save(f"data/ProcessedData/{dataset_name_num}/{new_path}", MIF)

In [52]:
def process_dataset():
    
    import os
    parent_dir = r"C:\Users\Linden\GitHub\3DQSAR\data\ProcessedData"
    new_dir_path = os.path.join(parent_dir, dataset_name_num)
    os.mkdir(new_dir_path)
    
    for entry in os.scandir(open3D_directory):
        if (entry.path.endswith(".agrd")):
            name = entry.path
            agrd_to_array(name)
    

In [57]:
def generate_labels():
    path = os.path.join(r"C:\Users\Linden\GitHub\3DQSAR\data\DatasetsEdited",dataset_name_num)
    tr = pd.read_csv(os.path.join(path,"pIC50.tr.txt"),header=None,delimiter=" ",names=["Name","Y"])
    if os.path.isfile(os.path.join(path,"pIC50.ts.txt")) == True:
        ts = pd.read_csv(os.path.join(path,"pIC50.ts.txt"),header=None,delimiter=" ",names=["Name","Y"])
        Y = pd.concat([tr,ts],ignore_index=True)
    else:
        Y = tr
    Y.sort_values(by=['Name'],inplace=True,ignore_index=True)
    count = len(Y)
    Y['ID']=np.arange(1,count+1)
    Y.to_csv(path_or_buf = os.path.join("data/ProcessedData/",
                                        dataset_name_num,
                                        "pIC50.csv"),
             index=False)

    
    

In [58]:
s=r"C:\open3dtools\therm_fld-01_obj-101.agrd"

In [59]:
if __name__=="__main__":
    set_globals()
    process_dataset()
    generate_labels()

In [ ]:
"""

import type=mol2 file="C:\Users\Linden\GitHub\3DQSAR\data\DatasetsEdited\02.AchE\Data\combined_AchE.sdf"

box step=1.0

calc_field type=VDW

calc_field type=MM_ELE

export type=OBJECT_FIELD file=therm format=XYZ 



"""